In [112]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from datetime import date
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [114]:
# Import data
file_path = Path("Resources/sp500_adj_close_raw.csv")
df = pd.read_csv(file_path)
# Convert date to datetime data type
df["Date"] = pd.to_datetime(df["Date"])
df.set_index('Date', inplace=True)
df.shape
df.tail()

,Ticker,Adjusted Close,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
Date,,,,,,,,,,,,,,,
2024-10-25,XYL,130.41,130.41,130.65,-0.001837,0.009297,39.056873,133.51048,134.41891,130.84920,138.399480,129.94153,126.71,137.53000,short
2024-10-25,YUM,133.04,133.04,133.16,-0.000901,0.006858,34.395523,134.61107,133.44423,134.00830,139.507830,130.66417,129.71,139.92000,short
2024-10-25,ZBH,102.33,102.33,104.00,-0.016058,0.010610,50.197327,107.36608,107.96970,115.37445,107.723175,101.30882,101.77,115.91237,short
2024-10-25,ZBRA,359.97,359.97,362.05,-0.005745,0.009788,43.769238,355.89080,336.89050,311.04135,380.011440,359.82355,320.77,377.68000,short
2024-10-25,ZTS,179.91,179.91,181.50,-0.008760,0.012576,36.496883,189.09400,183.31490,179.24370,197.887830,182.27017,179.91,196.48000,short


In [115]:
# Do some feature engineering
df['Year'] = df.index.year
df['Month'] = df.index.month
df['Day'] = df.index.day
df['Day_of_Week'] = df.index.dayofweek
df.tail()

,Ticker,Adjusted Close,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action,Year,Month,Day,Day_of_Week
Date,,,,,,,,,,,,,,,,,,,
2024-10-25,XYL,130.41,130.41,130.65,-0.001837,0.009297,39.056873,133.51048,134.41891,130.84920,138.399480,129.94153,126.71,137.53000,short,2024,10,25,4
2024-10-25,YUM,133.04,133.04,133.16,-0.000901,0.006858,34.395523,134.61107,133.44423,134.00830,139.507830,130.66417,129.71,139.92000,short,2024,10,25,4
2024-10-25,ZBH,102.33,102.33,104.00,-0.016058,0.010610,50.197327,107.36608,107.96970,115.37445,107.723175,101.30882,101.77,115.91237,short,2024,10,25,4
2024-10-25,ZBRA,359.97,359.97,362.05,-0.005745,0.009788,43.769238,355.89080,336.89050,311.04135,380.011440,359.82355,320.77,377.68000,short,2024,10,25,4
2024-10-25,ZTS,179.91,179.91,181.50,-0.008760,0.012576,36.496883,189.09400,183.31490,179.24370,197.887830,182.27017,179.91,196.48000,short,2024,10,25,4


In [118]:
# Initialize the LabelEncoder and fit it to the Action column:

le_action = LabelEncoder()

df['Action'] = le_action.fit_transform(df['Action'])

print(df)

           Ticker  Adjusted Close  Next Day Close  Previous Day Close  \
Date                                                                    
2008-01-02      A       23.256376       23.025747           23.538282   
2008-01-02   AAPL        5.876341        5.879056            5.974059   
2008-01-02    ABT       18.130210       18.019756           18.240660   
2008-01-02   ACGL        7.608889        7.764444            7.816667   
2008-01-02    ACN       26.437080       25.982517           26.415075   
...           ...             ...             ...                 ...   
2024-10-25    XYL      130.410000      130.410000          130.650000   
2024-10-25    YUM      133.040000      133.040000          133.160000   
2024-10-25    ZBH      102.330000      102.330000          104.000000   
2024-10-25   ZBRA      359.970000      359.970000          362.050000   
2024-10-25    ZTS      179.910000      179.910000          181.500000   

              Return  Volatility        RSI      S

In [75]:
# Define X and Y variables for train and test datasets
X = df.drop(columns=["Action", "Ticker", "Previous Day Close", "Resistance", "Upper Band", "SMA_50", "SMA_200", "Next Day Close"])
y = df["Action"]

# Preserve the Ticker column separately
tickers = df['Ticker']

In [77]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [79]:
# Create a StandardScaler model and fit it to the training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [120]:
# Define the parameter grid for GridSearchCV
param_grid = {'n_neighbors': np.arange(1, 30, 2)}


In [ ]:
# Initialize and run GridSearchCV
grid_knn = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy', verbose=2)
grid_knn.fit(X_train_scaled, y_train)


Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] END ......................................n_neighbors=1; total time=  34.7s
[CV] END ......................................n_neighbors=1; total time=  34.2s
[CV] END ......................................n_neighbors=1; total time=  34.3s
[CV] END ......................................n_neighbors=1; total time=  34.3s
[CV] END ......................................n_neighbors=1; total time=  34.1s
[CV] END ......................................n_neighbors=3; total time=  56.6s
[CV] END ......................................n_neighbors=3; total time=  57.0s
[CV] END ......................................n_neighbors=3; total time=  56.2s
[CV] END ......................................n_neighbors=3; total time=  57.5s
[CV] END ......................................n_neighbors=3; total time=  56.8s
[CV] END ......................................n_neighbors=5; total time= 1.2min


In [ ]:
# Best hyperparameters and corresponding score
print("Best Parameters:", grid_knn.best_params_)
print("Best Cross-Validation Score:", grid_knn.best_score_)


In [ ]:
# Fit the model with the best found hyperparameters
best_knn_classifier = grid_knn.best_estimator_
best_knn_classifier.fit(X_train_scaled, y_train)

In [ ]:
# Predict on the test set using the best model
y_pred_class = best_knn_classifier.predict(X_test_scaled)

In [ ]:
# Evaluate the classifier
print("Classification Report for Best KNN Classifier:")
print(classification_report(y_test, y_pred_class))

In [87]:
# Generate and display the confusion matrix
conf_matrix_class = confusion_matrix(y_test, y_pred_class)
conf_df_class = pd.DataFrame(
    conf_matrix_class, 
    index=le_action.classes_, 
    columns=le_action.classes_
)
print("Confusion Matrix:")
print(conf_df_class)

Confusion Matrix:
         buy   hold   sell  short
buy    67324   4068   1601  26069
hold    5294  69907  31323   1440
sell    2882  37218  57092   2286
short  33492   2542   3269  50313


In [95]:
# Predict actions for the last day and export to CSV:
last_day_class = df.loc[df.index == df.index.max()].drop(columns=["Action", "Previous Day Close", "Resistance", "Upper Band", "SMA_50", "SMA_200", "Next Day Close"])


In [97]:
# Preserve the Ticker column from the last day
last_day_tickers = last_day_class['Ticker']

In [99]:
# Drop the Ticker column before scaling
X_last_day_class = last_day_class.drop(columns=["Ticker"])

In [101]:
# Standardize the last day data
last_day_scaled = scaler.transform(X_last_day_class)
predicted_actions = knn_classifier.predict(last_day_scaled)


In [105]:
# Add predictions and ticker back to the DataFrame
last_day_class['Predicted_Action'] = le_action.inverse_transform(predicted_actions)
last_day_class['Ticker'] = last_day_tickers

In [107]:
# Display the last day's predictions with Ticker
print(last_day_class[['Ticker', 'Adjusted Close', 'Predicted_Action']])

           Ticker  Adjusted Close Predicted_Action
Date                                              
2024-10-25      A          130.16              buy
2024-10-25   AAPL          231.40             sell
2024-10-25   ABBV          187.82            short
2024-10-25   ABNB          134.60             hold
2024-10-25    ABT          114.24            short
...           ...             ...              ...
2024-10-25    XYL          130.41             sell
2024-10-25    YUM          133.04             hold
2024-10-25    ZBH          102.33            short
2024-10-25   ZBRA          359.97             sell
2024-10-25    ZTS          179.91              buy

[501 rows x 3 columns]


In [109]:
# Export the predictions
last_day_class.to_csv('predicted_actions_last_day.csv')